In [75]:
! pip install groq

In [76]:
import os
os.environ['GROQ_API_KEY'] = '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'

In [77]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be attributed to several factors:

1. **Real-time Applications**: Fast language models enable real-time processing and interaction with users, which is essential for applications like:
	* Conversational AI (e.g., chatbots, virtual assistants)
	* Sentiment analysis and feedback systems
	* Language translation and interpretation
2. **Scalability**: Fast language models can handle large volumes of data and support massive-scale NLP applications, such as:
	* Social media monitoring and analysis
	* Text classification and filtering for content moderation
	* Language models for question answering and recommendation systems
3. **Low-Latency Requirements**: Many NLP applications require rapid response times, making fast language models essential for:
	* Voice-activated assistants (e.g., Alexa, Google Assistant)
	* Real-time language translation for live events or meetings
	* Fast t

In [85]:
class Agent:
    
    # Initializes the agent
    def __init__(self, client, system):
        self.client = client
        # Initializes the system message containing the ReAct loop *Thought, Action, Observation*
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})
    
    # Called whenever the user calls on the agent    
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        
        return result
    
    def execute(self):    
        completion = client.chat.completions.create(
            messages=self.messages,
            model="llama3-70b-8192",
        )
        return completion.choices[0].message.content

        

In [79]:
# ReAct system prompt instructing the model how to iterate thouroughly until an answer is found
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


In [80]:
# tools accessible for the Agent
def calculate(operation):
    return eval(operation)

def get_planet_mass(planet: str) -> float:
    planet = planet.lower()
    if planet == "earth":
        return 5.972e24
    elif planet == "jupiter":
        return 1.898e27
    elif planet == "mars":
        return 6.39e23
    elif planet == "mercury":
        return 3.285e23
    elif planet == "neptune":
        return 1.024e26
    elif planet == "saturn":
        return 5.683e26
    elif planet == "uranus":
        return 8.681e25
    elif planet == "venus":
        return 4.867e24
    else:
        return 0.0

## Going through Agent Loop Manually

In [86]:
planet_expert = Agent(client=client, system=system_prompt)

In [87]:
result = planet_expert("what is the mass of the earth times 5?")
print(result)

Thought: I need to find the mass of Earth


In [88]:
planet_expert.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

In [90]:
result = planet_expert()
print(result)

Action: get_planet_mass: Earth
PAUSE


In [91]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [93]:
next_prompt = f"Observation: {observation}"
result = planet_expert(next_prompt)
print(result)

Action: calculate: 5.972e24 * 5
PAUSE


In [94]:
observation = calculate("5.972e24 * 5")
print(observation)

2.9860000000000004e+25


In [95]:
next_prompt = f"Observation: {observation}"
result = planet_expert(next_prompt)
print(result)

Answer: The mass of the earth times 5 is 2.9860000000000004e+25.


In [96]:
planet_expert.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the

## Automating Agent Loop

In [104]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ['calculate', 'get_planet_mass']
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)
        
        if "PAUSE" in result and "Action" in result:
            # Finds action name (function to use) and parameters
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]
            
            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"
                
        print(next_prompt)
        continue
        
        if "Answer" in result:
            break

In [105]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of the Earth plus the mass of Mars times 8?")

Thought: I need to find the mass of Earth and the mass of Mars.
What is the mass of the Earth plus the mass of Mars times 8?
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Mars and then multiply it by 8.
Observation: 5.972e+24
Action: get_planet_mass: Mars
PAUSE
Observation: 6.39e+23
Thought: I now have the mass of Mars, I need to multiply it by 8.
Observation: 6.39e+23
Action: calculate: 6.39e+23 * 8
PAUSE
Observation: 5.112e+24
Thought: Now I have the mass of Mars times 8, I need to add the mass of Earth.
Observation: 5.112e+24
Action: calculate: 5.972e+24 + 5.112e+24
PAUSE
Observation: 1.1084e+25
Thought: I have the final answer!

Answer: The mass of the Earth plus the mass of Mars times 8 is 1.1084e+25.
Observation: 1.1084e+25
Answer: The mass of the Earth plus the mass of Mars times 8 is 1.1084e+25.
Observation: 1.1084e+25
